<a href="https://colab.research.google.com/gist/davidbau/537afacb93ede5d83e80f1f0b8b9e249/colabnotebooksecrets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#What makes Colab so great?

In [0]:
%%bash
nvidia-smi

Tue May  5 15:28:00 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Yes, that's right.  Every Colab user gets their very own GPU, complements of Google!

The main disadvantage of colab it might not come with the exact environment you are used to.  But there are two reasons this is not really a problem.

1. The defaults are pretty reasonable.
2. You are root.

Check out the defaults: they include the latest versions of pytorch, tensorflow, java, gcc, etc.

In [0]:
%%bash

# As of this writing, colab gives us an Ubuntu 18.04 machine.
lsb_release -a

# It includes a whole pile of software, e.g., perl, python, java, gcc, git, etc
apt list --installed

# And it preinstalls a healthy set of python: pytorch, tensorflow, pandas, etc.
pip list

Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.3 LTS
Release:	18.04
Codename:	bionic
Listing...
adduser/bionic,now 3.116ubuntu1 all [installed]
adwaita-icon-theme/bionic,now 3.28.0-1ubuntu1 all [installed,automatic]
apt/bionic-updates,now 1.6.12 amd64 [installed]
apt-utils/bionic-updates,now 1.6.12 amd64 [installed]
base-files/now 10.1ubuntu2.7 amd64 [installed,upgradable to: 10.1ubuntu2.8]
base-passwd/bionic,now 3.5.44 amd64 [installed]
bash/bionic-updates,now 4.4.18-2ubuntu1.2 amd64 [installed]
binutils/bionic-updates,bionic-security,now 2.30-21ubuntu1~18.04.2 amd64 [installed,automatic]
binutils-common/bionic-updates,bionic-security,now 2.30-21ubuntu1~18.04.2 amd64 [installed,automatic]
binutils-x86-64-linux-gnu/bionic-updates,bionic-security,now 2.30-21ubuntu1~18.04.2 amd64 [installed,automatic]
blt/bionic,now 2.5.3+dfsg-4 amd64 [installed,automatic]
bsdmainutils/bionic,now 11.1.2ubuntu1 amd64 [installed,automatic]
bsdutils/now 1:2.31.1-0.4ubuntu3.4 amd64 [installed,upgradable to

No LSB modules are available.




And then check out the root access. You can install new packages at will.

In [0]:
%%bash
whoami

root


In [0]:
%%bash
apt-get install -q blender

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  blender-data fonts-dejavu fonts-dejavu-core fonts-dejavu-extra libblosc1
  libglew2.0 libjemalloc1 liblog4cplus-1.1-9 libopencolorio1v5
  libopenimageio1.7 libopenvdb5.0 libraw16 libspnav0 libtinyxml2.6.2v5
  libyaml-cpp0.5v5
Suggested packages:
  glew-utils spacenavd
The following NEW packages will be installed:
  blender blender-data fonts-dejavu fonts-dejavu-core fonts-dejavu-extra
  libblosc1 libglew2.0 libjemalloc1 liblog4cplus-1.1-9 libopencolorio1v5
  libopenimageio1.7 libopenvdb5.0 libraw16 libspnav0 libtinyxml2.6.2v5
  libyaml-cpp0.5v5
0 upgraded, 16 newly installed, 0 to remove and 25 not upgraded.
Need to get 45.2 MB of archives.
After this operation, 224 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 blender-data all 2.79.b+dfsg0-1ubuntu1.18.04.1 [12.2 MB]
Get:2 http://archive

# Colab Notebook Boostrapping

The main Google Colab disadvantage is that your data and environment are not there yet. So to make a Colab notebook, here is our plan of attack.
1. Make a standard Jupyter notebook in your own project, on your own machine.
2. Then make the notebook bring in its own dependencies when running on colab.

Once we have a working notebook, at the top all we need to do is include a small section that runs only if the notebook is running on Colab.  It can look something like this:



In [0]:
%%bash

# Exit this cell if not running in colab.
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
pip install ninja 2>> install.log
git clone https://github.com/SIDN-IAP/global-model-repr.git tutorial_code 2>> install.log

In [0]:
try: # this will alter the path if inside Google colab.
    import google.colab, sys, torch
    sys.path.append('/content/tutorial_code')
    if not torch.cuda.is_available():
        print("Change runtime type to include a GPU.")  
except:
    pass

# Other Google Colab Survival Tricks

1. Use torch.hub to download models and automatically cache them, so you do not need to redownload things.
2. Use a utility like `netdissect.show` for showing grids of images and other data.
3. Unlike juypter, every Colab cell runs on its own independent webpage (in an iframe), so normal widgets do not work.  Use `netdissect.labwidgets` instead.

See the following examples to show these techniques.

https://colab.research.google.com/github/SIDN-IAP/global-model-repr/blob/master/notebooks/gandissect_solutions.ipynb

https://colab.research.google.com/github/SIDN-IAP/interactivity/blob/master/notebooks/ganpaint-solution.ipynb

# Loading and Saving Data

There are two options to load data into Google Colab. You can either download it directly from a server using a url, or you can load it by connecting to your Google Drive.

The first option is easier and will save you Drive space. However, the data will not be persistent when you close the colab session.

In [0]:
import requests

# Download a file into the filename fn
def download(url, fn=None):
  if fn is None:
    fn = url.split('/')[-1]
  r = requests.get(url)
  if r.status_code == 200:
      open(fn, 'wb').write(r.content)
      print("{} downloaded: {:.2f} KB".format(fn, len(r.content)/1024.))
  else:
      print("url not found:", url)

In [0]:
download('http://6.869.csail.mit.edu/fa19/psets19/pset6/WelshCorgi.jpeg', 'dog.jpeg')
! ls .

dog.jpeg downloaded: 127.39 KB
dog.jpeg  drive  install.log  sample_data  tutorial_code


To use Google Drive, you need to run the following code, and enter the authorization code.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


You can access your data at `/content/drive/My Drive/`